In [1]:
import pandas as pd
import numpy as np
import os
import gc
import random

from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# set a seed value
torch.manual_seed(555)

import transformers
from transformers import TrainingArguments, Trainer
from transformers import EarlyStoppingCallback
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AdamW

In [2]:
MODEL_TYPE = 'indobenchmark/indobert-base-p1'
L_RATE = 1e-6
MAX_LEN = 512

NUM_EPOCHS = 6
BATCH_SIZE = 1
NUM_CORES = os.cpu_count() - 2

In [3]:
os.environ["WANDB_DISABLED"] = "true"

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [5]:
df_train = pd.read_csv("D:/Training/Machine Learning/Datasets/indonli-with-java-chatgpt-training-data.csv", sep='\t')
df_train = df_train.sample(frac=1).reset_index(drop=True) #shuffle the data

In [6]:
df_train_new = pd.DataFrame()
df_train_new["premise"] = df_train["premise"]
df_train_new["hypothesis"] = df_train["jv_hypothesis"]
df_train_new["label"] = df_train["label"]
df_train_new.head()

,premise,hypothesis,label
0,"Menurut Nur, bantuan segera yang diharapkan wa...",Cairan antikuman iku kanggo sing paling dibutu...,1
1,"Pada 30 Desember 1966, purwarupa ini memasuki ...",Purwarupa iki pepunden dadi anggota T-64.,0
2,"Menurut hitungan sutradara itu sendiri, hingga...","Sajrone sutradara iki, dheweke wis ndadeke wol...",1
3,"Pada tahun 1271, setelah sebulan pertempuran, ...",Baibar kuwi wonten ingkang dipunanggit sultan.,1
4,Mana yang harus dikerjakan lebih dulu dan mana...,"""Dedline tugas kuliah lan pitulungan kudu dipé...",0


In [7]:
df_valid = pd.read_csv("D:/Training/Machine Learning/Datasets/indonli-with-java-chatgpt-validation-data.csv", sep='\t')
df_valid = df_valid.sample(frac=1).reset_index(drop=True) #shuffle the data

In [8]:
df_valid_new = pd.DataFrame()
df_valid_new["premise"] = df_valid["premise"]
df_valid_new["hypothesis"] = df_valid["jv_hypothesis"]
df_valid_new["label"] = df_valid["label"]
df_valid_new.head()

,premise,hypothesis,label
0,Cenote angelita berada di wilayah hutan lebat ...,Cenote angelita ana ing rimbawan sing duwé kea...,0
1,Delapan tim bersaing pada putaran pertama musi...,Pemenang putaran pertama musim iki saka Newcas...,1
2,"Sementara itu, air es yang dingin, meski teras...",Air es sing reges krasa segar.,0
3,"Pada hari-hari terakhir perang, saat Pertempur...",Eva Braun yaiku istri kalihé Hitler.,1
4,"Ibukota Hindia Belanda, Batavia, direbut oleh ...",Prajurit Jepang wes sukses ngluwari Batavia si...,0


In [9]:
df_test = pd.read_csv("D:/Training/Machine Learning/Datasets/indonli-with-java-google-translate-testing-data.csv", sep='\t')
df_test = df_test.sample(frac=1).reset_index(drop=True) #shuffle the data

In [10]:
df_test_new = pd.DataFrame()
df_test_new["premise"] = df_test["premise"]
df_test_new["hypothesis"] = df_test["jv_hypothesis"]
df_test_new["label"] = df_test["label"]
df_test_new.head()

,premise,hypothesis,label
0,Nazi adalah kekuatan politik utama di Jerman s...,Pimpinan Nazi didakwa karo kejahatan marang ma...,0
1,"Pada 24 Oktober 2012, terjadi unjuk rasa di Ba...",Konfederasi Seluruh Serikat Pekerja Indonesia ...,0
2,"Karena itu pula, paket ini hanya bisa digunaka...",Rega kuota khusus luwih murah tinimbang rega k...,1
3,Sejak pemerintahan jatuh ke tangan China pada ...,Hong Kong ora dianggep manungsa kanthi manungs...,1
4,"Bahkan, Diesel tampil secara langsung untuk pe...",Billboard Latin Music Awards ora ngundang Dies...,2


In [11]:
tokenizer = BertTokenizer.from_pretrained(MODEL_TYPE)

In [12]:
class CompDataset(Dataset):
    def __init__(self, df):
        self.df_data = df
        
    def __getitem__(self, index):
        sentence1 = self.df_data.loc[index, 'premise']
        sentence2 = self.df_data.loc[index, 'hypothesis']
        
        encoded_dict = tokenizer.encode_plus(
            sentence1,
            sentence2,
            add_special_tokens = True,
            max_length = MAX_LEN,
            truncation='longest_first',
            padding = 'max_length',
            return_attention_mask = True,
            return_tensors = 'pt'
        )
        
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        
        target = torch.tensor(self.df_data.loc[index, 'label'])
        sample = {"input_ids": padded_token_list, "attention_mask": att_mask, "label": target}
        
        return sample
    
    def __len__(self):
        return len(self.df_data)
    

In [13]:
train_data_cmp = CompDataset(df_train_new)
valid_data_cmp = CompDataset(df_valid_new)
test_data_cmp = CompDataset(df_test_new)

In [14]:
print(f"Jumlah core: {str(NUM_CORES)}")

Jumlah core: 14


In [15]:
model = BertForSequenceClassification.from_pretrained(MODEL_TYPE, num_labels=3)

# Send the model to the device.
# model.to(device)

C:\Users\sufin\anaconda3\envs\my-torch\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sufin\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are 

In [16]:
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='micro')
    precision = precision_score(y_true=labels, y_pred=pred, average='micro')
    f1 = f1_score(y_true=labels, y_pred=pred, average='weighted')

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [17]:
training_args = TrainingArguments(
    output_dir="D:/Kuliah/Bimbingan Tesis/Indo-Javanese-NLI/BaselineModels/IndoBERT-base",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    eval_steps=1000,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
    seed=101,
    learning_rate=L_RATE
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data_cmp,
    eval_dataset=valid_data_cmp,
    compute_metrics=compute_metrics,
)

In [19]:
# Train pre-trained model
trainer.train()

C:\Users\sufin\anaconda3\envs\my-torch\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.111900,1.267631,0.539372,0.539372,0.539372,0.544325
2,1.657700,1.820758,0.562585,0.562585,0.562585,0.566522
3,1.635400,1.798471,0.573509,0.573509,0.573509,0.574010
4,1.537200,1.874364,0.578971,0.578971,0.578971,0.580081
5,1.729500,1.930055,0.573964,0.573964,0.573964,0.572187
6,1.566200,1.956391,0.580337,0.580337,0.580337,0.580568


TrainOutput(global_step=61980, training_loss=1.4649211970941987, metrics={'train_runtime': 9224.3697, 'train_samples_per_second': 6.719, 'train_steps_per_second': 6.719, 'total_flos': 1.630776963078144e+16, 'train_loss': 1.4649211970941987, 'epoch': 6.0})

In [20]:
# # Make prediction
prediction = trainer.predict(test_data_cmp) # test_trainer.predict(test_data_cmp)

In [21]:
print(prediction)

PredictionOutput(predictions=array([[-2.4369802 ,  3.924709  , -2.4742856 ],
       [ 0.72177696, -0.03155722, -1.4706799 ],
       [-2.8649588 ,  3.7732317 , -2.4242692 ],
       ...,
       [ 2.7672362 , -1.5160373 , -1.7842759 ],
       [-2.261733  ,  3.6666076 , -2.7998977 ],
       [ 4.332664  , -1.4563729 , -2.2134128 ]], dtype=float32), label_ids=array([0, 0, 1, ..., 2, 1, 0], dtype=int64), metrics={'test_loss': 2.0811593532562256, 'test_accuracy': 0.5701953657428441, 'test_precision': 0.5701953657428441, 'test_recall': 0.5701953657428441, 'test_f1': 0.5724141875878537, 'test_runtime': 65.1362, 'test_samples_per_second': 33.791, 'test_steps_per_second': 33.791})


In [22]:
# # Preprocess raw predictions
y_pred = np.argmax(prediction[0], axis=1)

In [23]:
def return_label(the_label):
    str_label = ""
    if str(the_label) == "0":
        str_label = "entail"
    elif str(the_label) == "1":
        str_label = "neutral"
    else:
        str_label = "contradict"
    return str_label

In [24]:
for idx, row in df_test_new.iterrows():
    ground_truth = prediction[1][idx]
    if y_pred[idx] != ground_truth:
        print("==========================================================================================")
        print(f"Premis: {row['premise']}") 
        print(f"Hipotesis: {row['hypothesis']}")
        print(f"True Label: {return_label(ground_truth)}") 
        print(f"Pred Label: {return_label(y_pred[idx])}")
        print("==========================================================================================")

Premis: Nazi adalah kekuatan politik utama di Jerman sejak kejatuhan Republik Weimar pada tahun 1933 hingga akhir Perang Dunia II pada tahun 1945. Ketika dideklarasikan ilegal, para pemimpinnya ditangkap dan dikenai tuduhan kejahatan terhadap kemanusiaan melalui Pengadilan Nurenberg.
Hipotesis: Pimpinan Nazi didakwa karo kejahatan marang manungsa ing Nuremberg Trials.
True Label: entail
Pred Label: neutral
Premis: Sejak pemerintahan jatuh ke tangan China pada 1 Juli 1997, unjuk rasa memperjuangkan kemerdekan Hong Kong terus berlangsung. Pada sidang kongres Mei lalu, Joshua Wong, aktivis pro-demokrasi yang masih berusia 20 tahun, menggelar sebuah kampanye bertema "Penganiayaan Politik Besar-besaran" dan hingga kini masih berlangsung di Hong Kong.
Hipotesis: Hong Kong ora dianggep manungsa kanthi manungsa dening China.
True Label: neutral
Pred Label: contradict
Premis: Air Tawar Timur adalah salah satu kelurahan di Kecamatan Padang Utara, Padang, Sumatera Barat, Indonesia.
Hipotesis: Sum